In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
%matplotlib inline

import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
import librosa

import torch
from torch.utils.data import DataLoader

from utils import log_textfile, get_mean_std, windows, pad_dimesions_mfcc, load_model
from basetrainer import BaseTrainer
from dataloader import load_data
from CustomDatasetMFCC import CustomDatasetMFCC
from models.resnet import ResNet, resnet34
from attacks import fgsm, pgd_linf

ModuleNotFoundError: No module named 'utils'

In [ ]:
DATADIR = '../../Data/'

data = load_data(DATADIR)

trainset = pd.read_csv('filenames/trainset_10_uids.csv')
valset = pd.read_csv('filenames/valset_10_uids.csv')
trainset = trainset.drop('Unnamed: 0', axis=1)
valset = valset.drop('Unnamed: 0', axis=1)

In [ ]:
trainset_dl = [tuple(x) for x in trainset.values]
valset_dl = [tuple(x) for x in valset.values]

train_filepaths = [i[2] for i in trainset_dl]
train_labels = [i[3] for i in trainset_dl]
valid_filepaths = [i[2] for i in valset_dl]
val_labels = [i[3] for i in valset_dl]

In [ ]:
mean=-3.121299957927269
std=50.02533504630946
batch_size=16
num_workers=8
data_train_sub = CustomDataset(train_filepaths, train_labels, mean, std)
data_valid_sub = CustomDataset(valid_filepaths, val_labels, mean, std)

train_data_loader = torch.utils.data.DataLoader(
    dataset=data_train_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

valid_data_loader = torch.utils.data.DataLoader(
    dataset=data_valid_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

In [ ]:
normal_model = resnet34(pretrained=False, progress=False).cuda()
normal_model.load_state_dict(torch.load('saved/10_uids_mfcc_resnet_norm_normal_10_epochs_128'))

X, y = next(iter(valid_data_loader))


delta_fgsm = fgsm(normal_model, X.float().cuda(), y.long().cuda(), epsilon=0.05, alpha=None, num_iter=None)

y_attack_pred = normal_model(X.float().cuda()+delta_fgsm.cuda())
y_pred = normal_model(X.float().cuda())


np.mean(torch.argmax(y_attack_pred, axis=-1).detach().cpu().numpy()==y.detach().cpu().numpy())

np.mean(torch.argmax(y_pred, axis=-1).detach().cpu().numpy()==y.detach().cpu().numpy())

torch.argmax(y_attack_pred, axis=-1)

In [ ]:
X_np = (X.float().cuda()).permute(0,2,3,1).detach().cpu().numpy()

X_np_2 = np.squeeze(X_np[1]).T
mean=-3.121299957927269
std=50.02533504630946
X_np_2 = X_np_2*std+mean

aa = librosa.feature.inverse.mfcc_to_mel(X_np_2, n_mels=128, dct_type=2, norm='ortho', ref=1.)
audio_file = librosa.feature.inverse.mel_to_audio(aa)

IPython.display.Audio(data=audio_file, rate=22050)